In [1]:
# !pip uninstall keras -y
# !pip uninstall keras-nightly -y
# !pip uninstall keras-Preprocessing -y
# !pip uninstall keras-vis -y
# !conda uninstall tensorflow -y

#!conda install tensorflow==2.3.1 -y
# !pip install keras==2.4



In [2]:
import tensorflow as tf

2022-07-20 10:36:38.754707: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-07-20 10:36:38.754721: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
tf.version.VERSION

'2.9.1'

In [3]:
import pandas as pd 
import os
import shutil
import cv2
import tensorflow as tf
from tensorflow.keras import Model
import tqdm

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import cv2
import numpy as np
import random
import time
# import tensorflow as tf
from yolov3.yolov4 import Create_Yolo
from yolov3.utils import detect_image
from yolov3.configs import *

In [5]:
tf.version.VERSION

'2.3.1'

## Directories

In [6]:
ROOT_DIR = os.getcwd()
LABEL_DIR = os.path.join(ROOT_DIR, 'pictor')
TRAIN_DIR = os.path.join(ROOT_DIR, 'pictor', 'pictor_train')
TEST_DIR = os.path.join(ROOT_DIR, 'pictor', 'pictor_test')
TRAIN_TXT = os.path.join(LABEL_DIR, 'pictor_ppe_crowdsourced_approach-01_train.txt')
TEST_TXT  = os.path.join(LABEL_DIR, 'pictor_ppe_crowdsourced_approach-01_test.txt')
CUSTOM_TRAIN_TXT = os.path.join(LABEL_DIR, 'pictor_train.txt')
CUSTOM_TEST_TXT = os.path.join(LABEL_DIR, 'pictor_test.txt')

In [7]:
print(os.listdir(LABEL_DIR))

['pictor_test', 'pictor_train', 'Images', 'pictor_train.txt', 'pictor_test.txt', 'pictor.names', 'pictor_test (copy)']


In [8]:
# train_images = pd.read_csv(os.path.join(LABEL_DIR, 'pictor_ppe_crowdsourced_approach-01_train.txt'), sep=' ')

In [9]:
# train_images.shape

***Readlines method***

In [10]:
# Get the image details
read_source_txt = False

if read_source_txt:
    # Read train label file contents
    with open(TRAIN_TXT) as trf:
        train_contents = trf.readlines()
    trf.close()

    # Read test label contents
    with open(TEST_TXT) as tsf:
        test_contents = tsf.readlines()
    tsf.close()


    # print(contents[0].split("\t"))

    train_image_list = []
    test_image_list = []

    for line in train_contents:
        train_image_list.append(line.split("\t")[0])

    for line in test_contents:
        test_image_list.append(line.split("\t")[0])


### Move image files 

In [11]:
# Source destination path

IMG_SRC_ROOT = os.path.join(ROOT_DIR, 'pictor', 'Images')
TRAIN_DEST_ROOT = os.path.join(ROOT_DIR, 'pictor', 'pictor_train')
TEST_DEST_ROOT = os.path.join(ROOT_DIR, 'pictor', 'pictor_test')

In [12]:
# Move train image

move_train = False

if move_train:
    for i in train_image_list:
        full_src_path = os.path.join(IMG_SRC_ROOT, i)
        full_dst_path = os.path.join(TRAIN_DEST_ROOT, i)
        shutil.copy(full_src_path, full_dst_path)

In [13]:
# Move test image
move_test = False
if move_test:
    for i in test_image_list:
        full_src_path = os.path.join(IMG_SRC_ROOT, i)
        full_dst_path = os.path.join(TEST_DEST_ROOT, i)
        shutil.copy(full_src_path, full_dst_path)

### Customize text file

In [14]:
customize_train_txt = False

if customize_train_txt:
    with open(CUSTOM_TRAIN_TXT, 'w') as trf:
        for cont in train_contents:
            sep_contents = cont.replace("\t", " ")
    # #         sep_contents_split = 
    #         print(sep_contents[0])
    #         break
            for i in sep_contents:
    #             trf.seek(0,0)
    #             trf.write("Hi"+i+"\n")
                trf.writelines(i)
    trf.close()
    with open(CUSTOM_TRAIN_TXT) as fp:
        lines = fp.read().splitlines()
    with open(CUSTOM_TRAIN_TXT, "w") as fp:
        for line in lines:
            print(TRAIN_DIR+"/"+line, file=fp)

In [15]:
customize_test_txt = False

if customize_test_txt:
    with open(CUSTOM_TEST_TXT, 'w') as trf:
        for cont in test_contents:
            sep_contents = cont.replace("\t", " ")
    # #         sep_contents_split = 
    #         print(sep_contents[0])
    #         break
            for i in sep_contents:
    #             trf.seek(0,0)
    #             trf.write("Hi"+i+"\n")
                trf.writelines(i)
    trf.close()

    with open(CUSTOM_TEST_TXT) as fp:
        lines = fp.read().splitlines()
    with open(CUSTOM_TEST_TXT, "w") as fp:
        for line in lines:
            print(TEST_DIR+"/"+line, file=fp)

# Load Model

In [16]:
def conver_to_h5(model):
    for layer in tqdm(model.layers):
        if layer.weights:
            weights = []
            for w in layer.weights:
                weight_name = os.path.basename(w.name).replace(':0', '')
                weight_file = layer.name + '_' + weight_name + '.npy'
                weight_arr = np.load(numpy_weight_file)

            # remove the "background class"
            if weight_file.startswith('Logits_bias'):
                weight_arr = weight_arr[1:]
            elif weight_file.startswith('Logits_kernel'):
                weight_arr = weight_arr[:, 1:]

            weights.append(weight_arr)
        layer.set_weights(weights)
    model.save_weights("Keras_model.h5")

In [17]:
while True:
    ID = random.randint(0, 200)
    label_txt = "pictor/pictor_test.txt"
    image_info = open(label_txt).readlines()[ID].split()

    image_path = image_info[0]

    #yolo = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=TRAIN_CLASSES)
    #yolo.load_weights(f"./checkpoints/{TRAIN_MODEL_NAME}") # use keras weights
    yolo_h5_model = tf.keras.models.load_model('pictor_trained_model.h5')
    detect_image(yolo_h5_model, image_path, "pitor_pred_test.jpg", input_size=YOLO_INPUT_SIZE, show=True, CLASSES=TRAIN_CLASSES, rectangle_colors=(255,0,0))
    break

ValueError: Unknown layer: SlicingOpLambda

In [6]:
while True:
    ID = random.randint(0, 200)
    label_txt = "pictor/pictor_test.txt"
    image_info = open(label_txt).readlines()[ID].split()

    image_path = image_info[0]

    yolo = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=TRAIN_CLASSES)
    yolo.load_weights(f"./checkpoints/{TRAIN_MODEL_NAME}") # use keras weights
    
    yolo.save("pictor_model.h5")
    # detect_image(yolo, image_path, "pitor_pred_test.jpg", input_size=YOLO_INPUT_SIZE, show=True, CLASSES=TRAIN_CLASSES, rectangle_colors=(255,0,0))
    break

In [16]:
y_model = tf.saved_model.load(os.path.join('home/zaman/Workspace/fractal/yolov3-python-lesson/TensorFlow-2.x-YOLOv3', 'pictor_model'))

OSError: SavedModel file does not exist at: home/zaman/Workspace/fractal/yolov3-python-lesson/TensorFlow-2.x-YOLOv3/pictor_model/{saved_model.pbtxt|saved_model.pb}

In [8]:
while True:
    ID = random.randint(0, 200)
    label_txt = "pictor/pictor_test.txt"
    image_info = open(label_txt).readlines()[ID].split()

    image_path = image_info[0]

    y_model = tf.keras.models.load_model("pictor_model")
    # detect_image(yolo, image_path, "pitor_pred_test.jpg", input_size=YOLO_INPUT_SIZE, show=True, CLASSES=TRAIN_CLASSES, rectangle_colors=(255,0,0))
    break

IndexError: list index out of range

## Check model saving techniques

In [7]:

# import required modules
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

In [8]:
# Load in the data
cifar10 = tf.keras.datasets.cifar10
 
# Distribute it to train and test set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
 
# Reduce pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0
 
# flatten the label values
y_train, y_test = y_train.flatten(), y_test.flatten()

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [9]:
# number of classes
K = len(set(y_train))
# calculate total number of classes for output layer
print("number of classes:", K)

# Build the model using the functional API
# input layer
i = Input(shape=x_train[0].shape)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
x = BatchNormalization()(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)
x = Dropout(0.2)(x)

# Hidden layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)

# last hidden layer i.e.. output layer
x = Dense(K, activation='softmax')(x)

model = Model(i, x)
model.summary()


number of classes: 10
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16

2022-07-20 10:45:44.645679: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-20 10:45:44.672791: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-20 10:45:44.672886: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-20 10:45:44.673505: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [10]:
# save model
model.save('./test-model-save-tech/gfgModel.h5')
print('Model Saved!')
 
# load model
savedModel=load_model('./test-model-save-tech/gfgModel.h5')
savedModel.summary()

Model Saved!
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)

In [11]:
# saving and loading the model weights
 
# save model
model.save_weights('./test-model-save-tech/gfgModelWeights')
print('Model Saved!')
 
# load model
savedModel = model.load_weights('./test-model-save-tech/gfgModelWeights')
print('Model Loaded!')

Model Saved!
Model Loaded!


In [12]:

# save model
model.save_weights('./test-model-save-tech/gfgModelWeights.h5')
print('Model Saved!')
 
# load model
savedModel = model.load_weights('./test-model-save-tech/gfgModelWeights.h5')
print('Model Loaded!')

Model Saved!
Model Loaded!


In [13]:
# SavedModel format

# Save model
# mobilenet_save_path = os.path.join("./test-model-save-tech/mobilenet/1/")
# os.mkdir("./test-model-save-tech/mobilenet/1/")
tf.saved_model.save(model, "./test-model-save-tech/savedmodel")

INFO:tensorflow:Assets written to: ./test-model-save-tech/savedmodel/assets


INFO:tensorflow:Assets written to: ./test-model-save-tech/savedmodel/assets


In [14]:
loaded = tf.saved_model.load("./test-model-save-tech/savedmodel")
print(list(loaded.signatures.keys()))  # ["serving_default"]

['serving_default']


In [16]:
loaded.summery()

AttributeError: '_UserObject' object has no attribute 'summery'